In [1]:
import numpy as np
import pandas as pd
import csv
import glob

import matplotlib.pyplot as plt

from scipy.stats import kstest
from scipy.stats import normaltest
from scipy.stats import norm

from itertools import combinations
import pyvinecopulib as pv

import seaborn as sns
from scipy import stats

In [2]:
# Assuming you have three different CSV files with file paths
file1 = r'CompassPaper/HKNW.csv'
file2 = r'CompassPaper/FACT.csv'
file3 = r'CompassPaper/HRYR.csv'

# Read the three CSV files into separate dataframes
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

# Get the column names
NBO_dat = df1.dropna()
CT_dat = df2.dropna()
KGL_dat = df3.dropna()

NBO_dat = NBO_dat.drop(NBO_dat.columns[0], axis=1)
CT_dat = CT_dat.drop(CT_dat.columns[0], axis=1)
KGL_dat = KGL_dat.drop(KGL_dat.columns[0], axis=1)


In [3]:
KGL_dat_tmx = KGL_dat[['t_max_24']]
NBO_dat_tmx = NBO_dat[['t_max_24']]
CT_dat_tmx = CT_dat[['t_max_24']]

In [4]:
appended_df = pd.concat([KGL_dat_tmx, NBO_dat_tmx, CT_dat_tmx], axis=1)
appended_df

,t_max_24,t_max_24,t_max_24
0,22.0,23.0,22.0
1,25.0,25.0,30.0
2,24.0,26.0,29.0
3,25.0,26.0,30.0
4,27.0,26.0,24.0
...,...,...,...
723,25.0,20.0,25.0
724,26.0,25.0,25.0
725,24.0,24.0,21.0
726,24.0,25.0,28.0


In [5]:
KGL_dat_tmx = KGL_dat_tmx.rename(columns={'t_max_24': 'col1'})
NBO_dat_tmx = NBO_dat_tmx.rename(columns={'t_max_24': 'col2'})
CT_dat_tmx = CT_dat_tmx.rename(columns={'t_max_24': 'col3'})


In [6]:
appended_df = pd.concat([KGL_dat_tmx, NBO_dat_tmx, CT_dat_tmx], axis=1)
appended_df

,col1,col2,col3
0,22.0,23.0,22.0
1,25.0,25.0,30.0
2,24.0,26.0,29.0
3,25.0,26.0,30.0
4,27.0,26.0,24.0
...,...,...,...
723,25.0,20.0,25.0
724,26.0,25.0,25.0
725,24.0,24.0,21.0
726,24.0,25.0,28.0


In [7]:
column_names = KGL_dat.columns.tolist()
# Transform copula data using the empirical distribution    
KGL_data = pv.to_pseudo_obs(KGL_dat)
KGL_data = pd.DataFrame(KGL_data, columns= column_names)
KGL_data.to_csv("KGL_data_Emp.csv", index=False)


In [8]:
# appended_df.corr()

In [9]:

def process_dataset(dataset):
    # get the column names
    columns = dataset.columns

    # get the combinations of the columns
    comb = combinations(columns, 2)

    # List of copulas to test
    copulas = [pv.Bicop(pv.BicopFamily.gaussian), pv.Bicop(pv.BicopFamily.student),
               pv.Bicop(pv.BicopFamily.clayton), pv.Bicop(pv.BicopFamily.gumbel),
               pv.Bicop(pv.BicopFamily.frank)]

    # Create an empty dataframe to store the results
    result_df = pd.DataFrame(columns=["col1", "col2", "BIC", "LogLikelihood", "AIC", "Copula", "BicopFamily"])

    for cols in comb:
        col1, col2 = cols
        # select the columns
        X = pd.to_numeric(dataset[col1], errors='coerce')
        Y = pd.to_numeric(dataset[col2], errors='coerce')
        Z = np.array(list(zip(X, Y)))  # Convert to numpy array

        # Transform copula data using the empirical distribution
        u = pv.to_pseudo_obs(Z)

        # initialize the best BIC, loglikelihood, AIC, and copula
        best_bic = float("inf")
        best_loglik = -float("inf")
        best_aic = float("inf")
        best_copula = None

        for copula in copulas:
            copula.fit(data=u)
            loglik = copula.loglik(u)
            aic = copula.aic(u)

            if aic < best_aic:
                best_aic = aic
                best_bic = copula.bic(u)
                best_loglik = loglik
                best_copula = copula

        # print the results of the best copula
        print("Columns: ", cols)
        print("BIC: ", best_bic)
        print("LogLikelihood: ", best_loglik)
        print("AIC: ", best_aic)
        print("Copula: ", best_copula)
        print("BicopFamily: ", best_copula.family)

        # Append the results to the dataframe
        result_df = result_df.append({"col1": col1, "col2": col2, "BIC": best_bic, "LogLikelihood": best_loglik,
                                      "AIC": best_aic, "Copula": str(best_copula), "BicopFamily": best_copula.family},
                                     ignore_index=True)

    return result_df


In [12]:
datasets = [NBO_dat, CT_dat, KGL_dat]  
# datasets = appended_df
# Process each dataset and store the results
results = []
for idx, dataset in enumerate(datasets):
    result = process_dataset(dataset)
    results.append(result)

    # Save the results to a CSV file
    result.to_csv(f"result_dataset_{idx+1}.csv", index=False)

    print(f"Results for Dataset {idx+1} saved to result_dataset_{idx+1}.csv.")
    print()
 


Columns:  ('t_max_24', 't_min_24')
BIC:  -132.20003724226922
LogLikelihood:  69.37982623988123
AIC:  -136.75965247976245
Copula:  <pyvinecopulib.Bicop>
Frank, parameters = 2.9019
BicopFamily:  BicopFamily.frank
Columns:  ('t_max_24', 'td_mean_24')
BIC:  -16.034781069923596
LogLikelihood:  11.29719815370842
AIC:  -20.59439630741684
Copula:  <pyvinecopulib.Bicop>
Gaussian, parameters = -0.182904
BicopFamily:  BicopFamily.gaussian
Columns:  ('t_max_24', 'ws_mean_24')
BIC:  -178.8132815134682
LogLikelihood:  92.68644837548072
AIC:  -183.37289675096144
Copula:  <pyvinecopulib.Bicop>
Clayton, parameters = 0.779503
BicopFamily:  BicopFamily.clayton
Columns:  ('t_max_24', 'c_mean_24')
BIC:  -410.8562998329997
LogLikelihood:  208.70795753524646
AIC:  -415.41591507049293
Copula:  <pyvinecopulib.Bicop>
Gaussian, parameters = -0.678036
BicopFamily:  BicopFamily.gaussian
Columns:  ('t_max_24', 'rh_mean_day')
BIC:  -607.9107371204245
LogLikelihood:  307.2351761789589
AIC:  -612.4703523579178
Copula:

Columns:  ('t_min_24', 'rh_mean_day')
BIC:  -26.081442466982217
LogLikelihood:  16.333807060918446
AIC:  -30.66761412183689
Copula:  <pyvinecopulib.Bicop>
Frank, parameters = -1.26116
BicopFamily:  BicopFamily.frank
Columns:  ('t_min_24', 'p_max_24')
BIC:  -484.60723936475927
LogLikelihood:  245.59670550980698
AIC:  -489.19341101961396
Copula:  <pyvinecopulib.Bicop>
Frank, parameters = -6.02092
BicopFamily:  BicopFamily.frank
Columns:  ('t_min_24', 'p_min_24')
BIC:  -294.27343452598615
LogLikelihood:  150.42980309042042
AIC:  -298.85960618084084
Copula:  <pyvinecopulib.Bicop>
Frank, parameters = -4.38402
BicopFamily:  BicopFamily.frank
Columns:  ('td_mean_24', 'ws_mean_24')
BIC:  -88.3580942695557
LogLikelihood:  47.472132962205194
AIC:  -92.94426592441039
Copula:  <pyvinecopulib.Bicop>
Clayton, parameters = 0.493432
BicopFamily:  BicopFamily.clayton
Columns:  ('td_mean_24', 'c_mean_24')
BIC:  -23.69226445136849
LogLikelihood:  15.139218053111582
AIC:  -28.278436106223165
Copula:  <pyv

Columns:  ('c_mean_24', 'rh_mean_day')
BIC:  -591.9091261394733
LogLikelihood:  299.2386020254426
AIC:  -596.4772040508852
Copula:  <pyvinecopulib.Bicop>
Gaussian, parameters = 0.757322
BicopFamily:  BicopFamily.gaussian
Columns:  ('c_mean_24', 'p_max_24')
BIC:  -3.683825264096468
LogLikelihood:  5.125951587754222
AIC:  -8.251903175508444
Copula:  <pyvinecopulib.Bicop>
Gaussian, parameters = -0.125391
BicopFamily:  BicopFamily.gaussian
Columns:  ('c_mean_24', 'p_min_24')
BIC:  5.219864533392091
LogLikelihood:  0.6741066890099425
AIC:  0.6517866219801149
Copula:  <pyvinecopulib.Bicop>
Gaussian, parameters = -0.0454821
BicopFamily:  BicopFamily.gaussian
Columns:  ('rh_mean_day', 'p_max_24')
BIC:  -24.42727176098845
LogLikelihood:  15.497674836200213
AIC:  -28.995349672400426
Copula:  <pyvinecopulib.Bicop>
Frank, parameters = -1.3014
BicopFamily:  BicopFamily.frank
Columns:  ('rh_mean_day', 'p_min_24')
BIC:  -1.6294546232544374
LogLikelihood:  4.098766267333207
AIC:  -6.197532534666413
Co

In [13]:
# Print or further process the results as needed
for idx, result in enumerate(results):
    print(f"Results for Dataset {idx+1}:")
    print(result)
   


Results for Dataset 1:
           col1         col2          BIC  LogLikelihood          AIC  \
0      t_max_24     t_min_24  -132.200037      69.379826  -136.759652   
1      t_max_24   td_mean_24   -16.034781      11.297198   -20.594396   
2      t_max_24   ws_mean_24  -178.813282      92.686448  -183.372897   
3      t_max_24    c_mean_24  -410.856300     208.707958  -415.415915   
4      t_max_24  rh_mean_day  -607.910737     307.235176  -612.470352   
5      t_max_24     p_max_24  -270.556935     138.558275  -275.116550   
6      t_max_24     p_min_24  -478.422996     242.491305  -482.982611   
7      t_min_24   td_mean_24  -192.877771      99.718693  -197.437386   
8      t_min_24   ws_mean_24  -159.411174      82.985395  -163.970789   
9      t_min_24    c_mean_24     1.209239       2.675188    -3.350377   
10     t_min_24  rh_mean_day    -4.455548       8.787389   -13.574779   
11     t_min_24     p_max_24  -267.989362     137.274488  -272.548977   
12     t_min_24     p_min_24